## Gradient Boost Trees

Este código es un ejemplo de cómo utilizar Spark y su librería de Machine Learning, PySpark ML, para entrenar y evaluar un modelo de clasificación de Gradient Boosted Trees en el conjunto de datos "Iris" de UCI.

En esta sección, importamos las librerías necesarias para trabajar con Spark y con los diferentes componentes que vamos a utilizar para entrenar y evaluar nuestro modelo.

In [ ]:
# Importamos las librerías necesarias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Aquí creamos una sesión de Spark con el nombre "GBTClassifier". Esta sesión nos permite trabajar con Spark y cargar, preparar y analizar grandes conjuntos de datos de manera distribuida.

In [ ]:


# Creamos una sesión de Spark
spark = SparkSession.builder.appName("GBTClassifier").getOrCreate()


En esta sección, cargamos el conjunto de datos "Iris" de UCI en un dataframe de Spark. Los datos se encuentran en un archivo CSV en línea, y los leemos usando el método read de SparkSession y especificando el formato del archivo, si tiene encabezados y si se debe inferir el esquema.

In [ ]:

# Cargamos el conjunto de datos (usaremos el conjunto de datos "Iris" de UCI)
data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")


En esta sección, seleccionamos las columnas que vamos a usar como variables de entrada en nuestro modelo y les asignamos nombres más descriptivos. Las columnas son "sepal_length", "sepal_width", "petal_length", "petal_width" y "species".

In [ ]:

# Seleccionamos las columnas necesarias y las renombramos
data = data.selectExpr("_c0 as sepal_length", "_c1 as sepal_width", "_c2 as petal_length", "_c3 as petal_width", "_c4 as species")


Aquí, creamos un transformador VectorAssembler que convierte nuestras variables de entrada en un único vector de características, que se usará como entrada para nuestro modelo.

In [ ]:

# Convertimos las variables de entrada en un vector de características
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
data = assembler.transform(data)


En esta sección, convertimos nuestra variable objetivo "species" en un índice numérico usando el transformador StringIndexer. Esto nos permite usar la variable en nuestro modelo.

In [ ]:

# Convertimos la variable objetivo en un índice numérico
labelIndexer = StringIndexer(inputCol="species", outputCol="label")
data = labelIndexer.fit(data).transform(data)


En esta sección, dividimos nuestro conjunto de datos en conjuntos de entrenamiento y prueba, con una proporción de 70% para entrenamiento y 30% para prueba. Usamos la función randomSplit de Spark y especificamos una semilla para reproducibilidad.

In [ ]:

# Dividimos el conjunto de datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=123)


En esta sección, se crea un modelo de Gradient Boosted Trees utilizando el GBTClassifier de la librería pyspark.ml.classification. labelCol especifica la columna que contiene los valores de la variable objetivo y featuresCol especifica la columna que contiene el vector de características. maxIter es el número máximo de iteraciones para el algoritmo.

In [ ]:

# Creamos un modelo de Gradient Boosted Trees
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)


Aquí, el modelo de Gradient Boosted Trees se entrena utilizando los datos de entrenamiento.

In [ ]:

# Entrenamos el modelo con el conjunto de entrenamiento
model = gbt.fit(trainingData)


En esta sección, se utiliza el método transform para realizar predicciones en el conjunto de prueba utilizando el modelo entrenado.

In [ ]:

# Obtenemos las predicciones con el conjunto de prueba
predictions = model.transform(testData)


Aquí, se utiliza MulticlassClassificationEvaluator de la librería pyspark.ml.evaluation para evaluar la precisión del modelo. labelCol especifica la columna que contiene los valores de la variable objetivo, predictionCol especifica la columna que contiene las predicciones y metricName especifica la métrica a utilizar para evaluar el modelo (en este caso, la precisión).

In [ ]:

# Evaluamos la precisión del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


Ahora se imprime la metrica seleccionada para evaluar este modelo, que es la precision.

In [ ]:

# Imprimimos la precisión del modelo
print("GBTClassifier - Precisión: {:.2f}%".format(accuracy*100))


Finalmente, se detiene la sesión de Spark.

In [ ]:

# Detenemos la sesión de Spark
spark.stop()